In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / '.env')

True

In [4]:
HOPSWORK_PROJECT_NAME = 'taxi_demand_ytjqdg'
HOPWSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [3]:
import pandas as pd
from datetime import datetime
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f"Downloading raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
2023-04 file is not available
2023-05 file is not available
2023-06 file is not available
2023-07 file is not available
2023-08 file is not available
2023-09 file is not available
2023-10 file is not available
2023-11 file is not available
2023-12 file is not available


In [4]:
rides

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
3403761,2023-03-31 23:24:25,163
3403762,2023-03-31 23:24:50,125
3403763,2023-03-31 23:26:31,50
3403764,2023-03-31 23:07:51,113


In [5]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:01<00:00, 149.40it/s]


In [6]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1
...,...,...,...
2893795,2023-03-31 19:00:00,6,265
2893796,2023-03-31 20:00:00,6,265
2893797,2023-03-31 21:00:00,11,265
2893798,2023-03-31 22:00:00,1,265


In [5]:
import hopsworks

In [6]:
project = hopsworks.login(
    project=HOPSWORK_PROJECT_NAME,
    api_key_value=HOPWSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255


In [7]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [12]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [13]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [12]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/47255/fs/46149/fg/50777


Uploading Dataframe: 0.00% |          | Rows 0/2893800 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/47255/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f6e38f15f30>, None)

Testando a leitura dos dados que subiram pra feature store

In [14]:
# Create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
import src.config as config

try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/47255/fs/46149/fv/time_series_hourly_feature_view/version/1


In [19]:
ts_data_, _ = feature_view.training_data(
    description='Time-series hourly taxi rides'
)

2023-05-22 19:49:50,741 INFO: USE `taxi_demand_ytjqdg_featurestore`
2023-05-22 19:49:51,150 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_ytjqdg_featurestore`.`time_series_hourly_feature_group_1` `fg0`


In [21]:
ts_data_.sort_values(by=['pickup_location_id', 'pickup_hour'])

,pickup_hour,rides,pickup_location_id
472769,2022-01-01 00:00:00,0,1
2271751,2022-01-01 01:00:00,0,1
112292,2022-01-01 02:00:00,0,1
2206952,2022-01-01 03:00:00,0,1
560174,2022-01-01 04:00:00,1,1
...,...,...,...
2429082,2023-03-31 19:00:00,6,265
577647,2023-03-31 20:00:00,6,265
2754646,2023-03-31 21:00:00,11,265
63898,2023-03-31 22:00:00,1,265


In [1]:
ts_data_, _ = feature_view.training_data(
    description='Time-series hourly taxi rides'
)
ts_data_.sort_values(by=['pickup_location_id', 'pickup_hour'])

NameError: name 'feature_view' is not defined